# Finite elasticity

## Introduction

This tutorial demonstrates how to setup and solve a nonlinear continuum mechanics problem using OpenCMISS-Iron in python. For the purpose of this tutorial, we will be solving a 3D finite elasticity problem. 

Provided that you have already installed the OpenCMISS library, the code excerpts in this section can be run interactively by entering it directly into the Python interpreter, which can be started by running python or ipython in a terminal. The code shown here can also be downloaded as a jupyter notebook in the following [link](https://github.com/OpenCMISS-Examples/basics_tutorial/blob/master/src/python/mechanics_tutorial.ipynb).

## Intended learning outcomes:
At the end of this tutorial you will:
* Know the steps involved in setting up and solving a nonlinear finite elasticity simulation with OpenCMISS-Iron.
* Know how different mechanical loads can be applied to a geometry.
* Know how information about the deformation can be extracted for analysis. 

## The finite elasticity, stress equilibrium equation. 
In this example we are solving the stress equilibrium equation for nonlinear finite elasticity. The equations are a member of the elasticity field equations set class and the Finite Elasticity type. The stress equilibrium equation represents the principle of linear momentum as follows:

$$\displaystyle \nabla \sigma + \rho \mathbf{b}-\rho \mathbf {a}=0. \qquad \text{in} \qquad  \Omega \\$$

where $\sigma$ is the Cauchy stress tensor, $\mathbf{b}$ is the body force vector and $\mathbf{a}$ is the vector representing the acceleration due to any unbalanced forces. The boundary equations for the stress equilibrium equation are partitioned into the Dirichlet boundary conditions representing a fixed displacement $u_d$ over the boundary $\Gamma_d$, and the Neumann boundary conditions representing the traction forces $\mathbf{t}$ applied on the boundary $\Gamma_t$ along the normal $\mathbf{n}$ as follows:

$$
\begin{aligned}
\displaystyle u &= u_d \quad &\text{on} \quad \Gamma_d \\
\displaystyle \sigma^T \mathbf{n} &= \mathbf{t} \quad &\text{on} \quad \Gamma_t 
\end{aligned}
$$

In this example we will solve the stress equilibrium equation for an incompressible material, which we shall perform using **the incompressibility constraint**: 

$$\displaystyle (I_3-3) = 0$$



## The problem being solved in this tutorial. 
This tutorial has been set up to solve the deformation of an isotropic, unit cube and to analyse large deformation kinematics in the cube. 

The problem we are about to solve includes 4 dependent variables. The first three variables represent each of the 3D coordinates $(x,y,z)$ of the mesh nodes in the deformed state. The incompressibility constraint is satisfied using lagrange multipliers, which are represented as a scalar variable, $p$. This variable is often referred to as the **hydrostatic pressure**.

Another important set of equations that need to be included when solving a mechanics problem are the stress-strain relationships or **constitutive relationships**. There are a set of constitutive equations that have already been included within the OpenCMISS-Iron library as [EquationSet subtypes](http://cmiss.bioeng.auckland.ac.nz/OpenCMISS/doc/user/group___o_p_e_n_c_m_i_s_s___equations_set_subtypes.html). We will demonstrate how these constants are used to incorporate the constitutive equation in the simulation below when setting up the equation set.

We will see in this tutorial how five different types of mechanical loads can be applied as follows:  

* Model 1 (Uniaxial extension of a unit cube)
* Model 2 (Equibiaxial extension of a unit cube)
* Model 3 (Simple shear of a unit cube)
* Model 4 (Shear of a unit cube)
* Model 5 (Extension and shear of a unit cube)


## Loading the OpenCMISS-Iron library
In order to use OpenCMISS we have to first import the opencmiss.iron module from the opencmiss package.

In [1]:
import numpy

# Intialise OpenCMISS
from opencmiss.iron import iron

## Set up parameters
The following set of variables are used in the tutorial and are defined here. There is a common practice among expert users to set these variables up at the start because we can then change these easily and re-run the rest of the code if we want to. 

Of note is **UsePressureBasis**. This is a boolean variable that we can set to true to include the incompressibility constraint or set to false if the material we want to represent in the model is nearly incompressible or compressible. 

**numberOfLoadIncrements** sets the number of load steps to be taken to solve the nonlinear mechanics problem. If you have not solved nonlinear mechanics problems before now would be a good time to get familiar with that before you deep dive into running nonlinear mechanics simulations. Here, we briefly explain the concept. 

**InterpolationType** is an integer variable that can be changed to choose one of the nine basis interpolation types defined in OpenCMISS-Iron [Basis Interpolation Specifications Constants](http://opencmiss.org/documentation/apidoc/iron/latest/python/classiron_1_1_basis_interpolation_specifications.html)

The weak form finite element matrix equations for the stress equilibrium equations above have been implemented in the OpenCMISS-Iron libraries as an elasticity equation subtype. Numerical treatment of the equations will show that the equations are nonlinear and, specifically, the determination of the deformed configuration coordinates turn out to be like a root finding exercise. We need to find the deformed coordinates $\mathbf{x}$ such that $f(x)=0$. These equations are therefore solved using a nonlinear iterative solver, **Newton Raphson technique** to be exact. Nonlinear solvers require an initial guess at the root of the equation and if we are far away from the actual solution then the solvers can diverge and give spurious $x$ values. Therefore, in most simulations it is best to split an entire mechanical load into lots of smaller loads. This is what numberOfLoadIncrements allows us to do. You will see it come to use in the control loop section of the tutorial below. 

In [2]:
# Set constants
X, Y, Z = (1, 2, 3)

#set model boundary condition setting to be used
model = 1
# Set dimensions of the cube
width = 1.0
length = 1.0
height = 1.0

UsePressureBasis = False
NumberOfGaussXi = 2
InterpolationType = 1
numberOfXi = 3


numberOfLoadIncrements = 1
numberGlobalXElements = 1
numberGlobalYElements = 1
numberGlobalZElements = 1


Assuming OpenCMISS has been correctly built with the Python bindings by
following the instructions in the programmer documentation, we can now access
all the OpenCMISS functions, classes and constants under the iron namespace.

The next section describes how we can interact with the OpenCMISS-Iron library
through an object-oriented API.

## Step by step guide

### 1. Creating a coordinate system

First we construct a coordinate system that will be used to describe the geometry in our problem. The 3D geometry will exist in a 3D space, so we need a 3D coordinate system.

In [3]:
# Create a 3D rectangular cartesian coordinate system
coordinateSystemUserNumber = 1
coordinateSystem = iron.CoordinateSystem()
coordinateSystem.CreateStart(coordinateSystemUserNumber)
coordinateSystem.DimensionSet(3)
coordinateSystem.CreateFinish()

### 2. Creating basis functions

The finite element description of our fields requires a basis function to interpolate field values over elements. In the cell below, we have set up the code such that you can choose from a number of basis function types for this problem. These have been defined as OpenCMISS-Iron [Basis Interpolation Specifications Constants](http://opencmiss.org/documentation/apidoc/iron/latest/python/classiron_1_1_basis_interpolation_specifications.html). The **InterpolationType** variable that was set at the top of the tutorial will be used to determine which basis interpolation will be used in this simulation.

We have also set up a second **pressureBasis** function for the hydrostatic pressure. In mechanics theory it is well understood that the interpolation scheme for the pressure basis should be one order lower than the geometric basis. **UsePressureBasis**, which is set at the top of the tutorial determines whether the incompressibility constraint will be used in the simulation or not. If the pressure basis is not defined then the simulation is set up for either nearly incompressible constitutive equations or for constitutive equations that describe the mechanical properties of compressible solid materials. 



In [4]:
# Define basis
basisUserNumber = 1
pressureBasisUserNumber = 2

basis = iron.Basis()
basis.CreateStart(basisUserNumber)
if InterpolationType in (1,2,3,4):
    basis.type = iron.BasisTypes.LAGRANGE_HERMITE_TP
elif InterpolationType in (7,8,9):
    basis.type = iron.BasisTypes.SIMPLEX
basis.numberOfXi = numberOfXi
basis.interpolationXi = (
    [iron.BasisInterpolationSpecifications.LINEAR_LAGRANGE]*numberOfXi)
if(NumberOfGaussXi>0):
    basis.quadratureNumberOfGaussXi = [NumberOfGaussXi]*numberOfXi
basis.CreateFinish()

if(UsePressureBasis):
    # Define pressure basis
    pressureBasis = iron.Basis()
    pressureBasis.CreateStart(pressureBasisUserNumber)
    if InterpolationType in (1,2,3,4):
        pressureBasis.type = iron.BasisTypes.LAGRANGE_HERMITE_TP
    elif InterpolationType in (7,8,9):
        pressureBasis.type = iron.BasisTypes.SIMPLEX
    pressureBasis.numberOfXi = numberOfXi
    pressureBasis.interpolationXi = (
        [iron.BasisInterpolationSpecifications.LINEAR_LAGRANGE]*numberOfXi)
    if(NumberOfGaussXi>0):
        pressureBasis.quadratureNumberOfGaussXi = [NumberOfGaussXi]*numberOfXi
    pressureBasis.CreateFinish()


### 3. Creating a region

Next we create a region that our fields will be defined on and tell it to use the 3D coordinate system we created previously. 

In [5]:
# Create a region and assign the coordinate system to the region
regionUserNumber = 1
region = iron.Region()
region.CreateStart(regionUserNumber,iron.WorldRegion)
region.LabelSet("Region")
region.coordinateSystem = coordinateSystem
region.CreateFinish()

### 4. Setting up a simple cuboid mesh

In this example we will use the GeneratedMesh class capabilities of OpenCMISS to create a 3D geometric mesh on which to solve the mechanics problem. We will create a regular mesh of size width x height y and depth z and divide the mesh into `numberGlobalXElements` in the X direction, `numberGlobalYElements` in the Y direction and  `numberGlobalZElements` in the Z direction. We will then tell it to use the basis we created previously:

In [6]:

# Start the creation of a generated mesh in the region
generatedMeshUserNumber = 1
generatedMesh = iron.GeneratedMesh()
generatedMesh.CreateStart(generatedMeshUserNumber,region)
generatedMesh.type = iron.GeneratedMeshTypes.REGULAR
if(UsePressureBasis):
    generatedMesh.basis = [basis,pressureBasis]
else:
    generatedMesh.basis = [basis]
    generatedMesh.extent = [width,length,height]
    generatedMesh.numberOfElements = (
        [numberGlobalXElements,numberGlobalYElements,numberGlobalZElements])
# Finish the creation of a generated mesh in the region
meshUserNumber = 1
mesh = iron.Mesh()
generatedMesh.CreateFinish(meshUserNumber,mesh)


### 5. Decomposing the mesh
Once the mesh has been created we can decompose it into a number of domains in order to allow for parallelism. We choose the options to let OpenCMISS calculate the best way to break up the mesh. We also set the number of domains to be equal to the number of computational nodes this example is running on. Note that if MPI infrastructure is not used, only single domain will be created. Look for our parallelisation example for an illustration of how to execute simulations using parallel processing techniques. 

In [7]:
# Get the number of computational nodes and this computational node number
numberOfComputationalNodes = iron.ComputationalNumberOfNodesGet()
# computationalNodeNumber = iron.ComputationalNodeNumberGet()

# Create a decomposition for the mesh
decompositionUserNumber = 1
decomposition = iron.Decomposition()
decomposition.CreateStart(decompositionUserNumber,mesh)
decomposition.type = iron.DecompositionTypes.CALCULATED
decomposition.numberOfDomains = numberOfComputationalNodes
decomposition.CreateFinish()

### 6. Creating a geometric field
Now that the mesh has been decomposed we are in a position to create fields. The first field we need to create is the geometric field. Here we create a field and partition the field to different computational nodes using the mesh decomposition that we have just created. Once we have finished creating the field we can change the field DOFs to give us our geometry. Since the mesh has been generated we can use the generated mesh object to calculate the geometric parameters of the regular mesh.

In [8]:
# Create a field for the geometry
geometricFieldUserNumber = 1

geometricField = iron.Field()
geometricField.CreateStart(geometricFieldUserNumber,region)
geometricField.MeshDecompositionSet(decomposition)
geometricField.TypeSet(iron.FieldTypes.GEOMETRIC)
geometricField.VariableLabelSet(iron.FieldVariableTypes.U,"Geometry")
geometricField.ComponentMeshComponentSet(iron.FieldVariableTypes.U,1,1)
geometricField.ComponentMeshComponentSet(iron.FieldVariableTypes.U,2,1)
geometricField.ComponentMeshComponentSet(iron.FieldVariableTypes.U,3,1)
if InterpolationType == 4:
    geometricField.fieldScalingType = iron.FieldScalingTypes.ARITHMETIC_MEAN
geometricField.CreateFinish()

# Update the geometric field parameters from generated mesh
generatedMesh.GeometricParametersCalculate(geometricField)

### 7. Creating a fibre field.

The mechanical behaviour of tissues and solids is fundamentally dependent on the microstructural organisation of the constituents that make up the tissue. With OpenCMISS-Iron one can define the anisotropic microstructural organisation using a 'fibre field'. The term **fibre** is a legacy terminology referring to the microstructural organisation of muscle fibres in the heart. Basically, the fibre field can be used to define local changes in the orientation of the microstructure of the tissue. This organisation is then used to create tensors that map important mechanics variables like stress and strain between the reference, deformed and microstructure coordinate systems. 

In [9]:
# Create a fibre field and attach it to the geometric field
fibreFieldUserNumber = 2

fibreField = iron.Field()
fibreField.CreateStart(fibreFieldUserNumber,region)
fibreField.TypeSet(iron.FieldTypes.FIBRE)
fibreField.MeshDecompositionSet(decomposition)
fibreField.GeometricFieldSet(geometricField)
fibreField.VariableLabelSet(iron.FieldVariableTypes.U,"Fibre")
if InterpolationType == 4:
    fibreField.fieldScalingType = iron.FieldScalingTypes.ARITHMETIC_MEAN #this is something specific to cubic-Hermites. 
fibreField.CreateFinish()

### Before moving to equations, for later use
Before setting up the equations set and equations below is a code snippet that we will use later to store the deformed configuration of the microstructural fibre field after the mechanics simulation is complete. 

In [10]:
# Create a deformed geometry field, as Cmgui/Zinc doesn't like displaying
# deformed fibres from the dependent field because it isn't a geometric field.
dependentFieldUserNumber = 4
deformedFieldUserNumber = 7
deformedField = iron.Field()
deformedField.CreateStart(deformedFieldUserNumber, region)
deformedField.MeshDecompositionSet(decomposition)
deformedField.TypeSet(iron.FieldTypes.GEOMETRIC)
deformedField.VariableLabelSet(iron.FieldVariableTypes.U, "DeformedGeometry")
for component in [1, 2, 3]:
    deformedField.ComponentMeshComponentSet(
            iron.FieldVariableTypes.U, component, 1)
if InterpolationType == 4:
    deformedField.ScalingTypeSet(iron.FieldScalingTypes.ARITHMETIC_MEAN)
deformedField.CreateFinish()

### 8. Create a pressure field.
In the next code snippet we create a pressure field. Remember that UsePressureBasis can be set to true or false to switch between compressible and incompressible elasticity. Here we just create the pressure field and let the logical switch variable determine whether the incompressibility equation is included in the problem set up or not. This pressure field stores the hydrostatic pressure variable value for the simulation. 

One can either make the hydrostatic pressure be only variable across elements as defined by the opencmiss-iron constant **iron.FieldInterpolationTypes.ELEMENT_BASED** below or variable across the nodes by using the OpenCMISS-Iron constant **iron.FieldInterpolationTypes.NODE_BASED**.

In [11]:
pressureFieldUserNumber = 8

pressureField = iron.Field()
pressureField.CreateStart(pressureFieldUserNumber, region)
pressureField.MeshDecompositionSet(decomposition)
pressureField.VariableLabelSet(iron.FieldVariableTypes.U, "Pressure")
pressureField.ComponentMeshComponentSet(iron.FieldVariableTypes.U, 1, 1)
pressureField.ComponentInterpolationSet(iron.FieldVariableTypes.U,1,
                                            iron.FieldInterpolationTypes.ELEMENT_BASED)
pressureField.NumberOfComponentsSet(iron.FieldVariableTypes.U, 1)
pressureField.CreateFinish()


### 9. Defining a equation set field, including the choice of constitutive equation.
We also need to create a new field called the equation set field.

Note the key constants used to define the equation set: 
**ProblemClasses.ELASTICITY** defines that the equation set is of the elasticity class.
**ProblemTypes.FINITE_ELASTICITY** defines that the finite elasticity equations set will be used.
**EquationsSetSubTypes.MOONEY_RIVLIN** defines that the mooney rivlin constitutive equation that has been defined within the OpenCMISS-Iron library should be used.

You can find more constitutive equation constants in the OpenCMISS [Equations Set Subtypes Constants](http://opencmiss.org/documentation/apidoc/iron/latest/python/classiron_1_1_equations_set_subtypes.html).


In [12]:
# Create the equations_set

equationsSetUserNumber = 1
equationsSetFieldUserNumber = 5

equationsSetField = iron.Field()
equationsSet = iron.EquationsSet()

problemSpecification = [iron.ProblemClasses.ELASTICITY,
    iron.ProblemTypes.FINITE_ELASTICITY,
    iron.EquationsSetSubtypes.MOONEY_RIVLIN]
equationsSet.CreateStart(equationsSetUserNumber,region,fibreField,problemSpecification,
    equationsSetFieldUserNumber, equationsSetField)
equationsSet.CreateFinish()

### 10. Defining the dependent field
For the mechanics equation we need a dependent field (our solution) to describe our dependent variables. Note that the dependent field has been pre-defined in the OpenCMISS-Iron library to contain four components when the finite elasticity, mooney rivlin subtype equations set is used. The first three components store the deformed coordinates and the fourth stores the pressure. In the code snippet below the fourth variable is set to only be variable across elements for all basis interpolation types except for cubic Hermites. In this case the interpolation scheme for the pressure field is node based. 



In [13]:
# Create the dependent field
dependentField = iron.Field()
equationsSet.DependentCreateStart(dependentFieldUserNumber,dependentField)
dependentField.VariableLabelSet(iron.FieldVariableTypes.U,"Dependent")
dependentField.ComponentInterpolationSet(iron.FieldVariableTypes.U,4,iron.FieldInterpolationTypes.ELEMENT_BASED)
dependentField.ComponentInterpolationSet(iron.FieldVariableTypes.DELUDELN,4,iron.FieldInterpolationTypes.ELEMENT_BASED)
if(UsePressureBasis):
  # Set the pressure to be nodally based and use the second mesh component
  if InterpolationType == 4:
    dependentField.ComponentInterpolationSet(iron.FieldVariableTypes.U,4,iron.FieldInterpolationTypes.NODE_BASED)
    dependentField.ComponentInterpolationSet(iron.FieldVariableTypes.DELUDELN,4,iron.FieldInterpolationTypes.NODE_BASED)
    dependentField.ComponentMeshComponentSet(iron.FieldVariableTypes.U,4,2)
    dependentField.ComponentMeshComponentSet(iron.FieldVariableTypes.DELUDELN,4,2)
    dependentField.fieldScalingType = iron.FieldScalingTypes.ARITHMETIC_MEAN
equationsSet.DependentCreateFinish()

### 11. Create the material field to store the constitutive equation parameter values. 
Below we set up a field called a material field, which will store the constitutive equation parameters that have been defined for the Mooney Rivlin equation subtype in OpenCMISS-Iron. This field can be set to have the same values throughout the mesh to represent a homogeneous material. If you want to describe heterogeneous materials you can set the values of these parameters differently across the mesh. This is not shown here but we'll give you a little example of this at the end of this tutorial. Below, the ```ComponentValuesInitialiseDP``` function sets all the nodal values to be the same: 1.0 for c10 and 0.2 for c01.

In [14]:
# Create the material field
materialFieldUserNumber = 3

materialField = iron.Field()
equationsSet.MaterialsCreateStart(materialFieldUserNumber,materialField)
materialField.VariableLabelSet(iron.FieldVariableTypes.U,"Material")
equationsSet.MaterialsCreateFinish()

# Set Mooney-Rivlin constants c10 and c01 respectively.
materialField.ComponentValuesInitialiseDP(
    iron.FieldVariableTypes.U,iron.FieldParameterSetTypes.VALUES,1,1.0)
materialField.ComponentValuesInitialiseDP(
    iron.FieldVariableTypes.U,iron.FieldParameterSetTypes.VALUES,2,0.2)


### 12. Create the equations from the equations set and material parameters.

Once the equations set is defined, we create the equations that use our fields to construct equations matrices and vectors.

In [15]:
# Create equations
equations = iron.Equations()
equationsSet.EquationsCreateStart(equations)
equations.sparsityType = iron.EquationsSparsityTypes.SPARSE
equations.outputType = iron.EquationsOutputTypes.NONE
equationsSet.EquationsCreateFinish()


### 13. Initialise the dependent field.
The dependent field needs to be initialised before the simulation is run. To this end, we copy the values of the coordinates from the geometric field into the dependent field in the below code snippet. The hydroastatic pressure field is set to 0.0.

In [16]:
# Initialise dependent field from undeformed geometry and displacement bcs and set hydrostatic pressure
iron.Field.ParametersToFieldParametersComponentCopy(
    geometricField,iron.FieldVariableTypes.U,iron.FieldParameterSetTypes.VALUES,1,
    dependentField,iron.FieldVariableTypes.U,iron.FieldParameterSetTypes.VALUES,1)
iron.Field.ParametersToFieldParametersComponentCopy(
    geometricField,iron.FieldVariableTypes.U,iron.FieldParameterSetTypes.VALUES,2,
    dependentField,iron.FieldVariableTypes.U,iron.FieldParameterSetTypes.VALUES,2)
iron.Field.ParametersToFieldParametersComponentCopy(
    geometricField,iron.FieldVariableTypes.U,iron.FieldParameterSetTypes.VALUES,3,
    dependentField,iron.FieldVariableTypes.U,iron.FieldParameterSetTypes.VALUES,3)
iron.Field.ComponentValuesInitialiseDP(
    dependentField,iron.FieldVariableTypes.U,iron.FieldParameterSetTypes.VALUES,4,0.0)

### 14. Defining the problem

Now that we have defined all the equations we will need we can create our problem to be solved by OpenCMISS. We create a standard finite elasticity problem, which is a member of the elasticity field problem class. The problem control loop uses the default load increment loop and hence does not require a subtype.  

In [17]:
# Define the problem
problemUserNumber = 1
problem = iron.Problem()
problemSpecification = [iron.ProblemClasses.ELASTICITY,
        iron.ProblemTypes.FINITE_ELASTICITY,
        iron.ProblemSubtypes.NONE]
problem.CreateStart(problemUserNumber, problemSpecification)
problem.CreateFinish()

### 15. Defining control loops

The problem type defines a control loop structure that is used when solving the problem. The OpenCMISS control loop is a "supervisor" for the computational process. We may have multiple control loops with nested sub loops, and control loops can have different types, for example load incremented loops or time loops for dynamic problems. These control loops have been defined in the OpenCMISS-Iron library for the finite elasticity type of equations as a load increment loop. If we wanted to access the control loop and modify it we would use the problem.ControlLoopGet method before finishing the creation of the control loops. In the below code snippet we get the control loop to set the number of load increments to be used to solve the problem using the variable **numberOfLoadIncrements**. 

In [18]:
# Create the problem control loop
problem.ControlLoopCreateStart()
controlLoop = iron.ControlLoop()
problem.ControlLoopGet([iron.ControlLoopIdentifiers.NODE],controlLoop)
controlLoop.MaximumIterationsSet(numberOfLoadIncrements)
problem.ControlLoopCreateFinish()


### 16. Defining solvers

After defining the problem structure we can create the solvers that will be run to actually solve our problem. The problem type defines the solvers to be set up so we call problem.SolversCreateStart() to create the solvers and then we can access the solvers to modify their properties. 

In [19]:
# Create problem solver
nonLinearSolver = iron.Solver()
linearSolver = iron.Solver()
problem.SolversCreateStart()
problem.SolverGet([iron.ControlLoopIdentifiers.NODE],1,nonLinearSolver)

nonLinearSolver.outputType = iron.SolverOutputTypes.NONE
nonLinearSolver.NewtonJacobianCalculationTypeSet(
        iron.JacobianCalculationTypes.EQUATIONS)
nonLinearSolver.NewtonLinearSolverGet(linearSolver)
linearSolver.linearType = iron.LinearSolverTypes.DIRECT
#linearSolver.libraryType = iron.SolverLibraries.LAPACK
problem.SolversCreateFinish()

### 17. Defining solver equations
After defining our solver we can create the equations for the solver to solve by adding our equations sets to a solver equations object. In this example we have just one equations set to add but for coupled problems we may have multiple equations sets in the solver equations. 

In [20]:
# Create solver equations and add equations set to solver equations
solver = iron.Solver()
solverEquations = iron.SolverEquations()
problem.SolverEquationsCreateStart()
problem.SolverGet([iron.ControlLoopIdentifiers.NODE],1,solver)
solver.SolverEquationsGet(solverEquations)
solverEquations.sparsityType = iron.SolverEquationsSparsityTypes.SPARSE
_ = solverEquations.EquationsSetAdd(equationsSet)
problem.SolverEquationsCreateFinish()


### 18. Defining the boundary conditions

The final step in configuring the problem is to define the boundary conditions to be satisfied. Here, as stated at the top of the tutorial we have set up five different boundary conditions settings to represent four different loading conditions on the cube geometry: 
* Model 1 (Uniaxial extension of a unit cube)
* Model 2 (Equibiaxial extension of a unit cube)
* Model 3 (Simple shear of a unit cube)
* Model 4 (Shear of a unit cube)
* Model 5 (Extension and shear of a unit cube)

The variable **model** set at the top of the tutorial program can be used to switch between these deformations. Each line of boundary condition code below sets dirchlet boundary conditions that prescribe a nodal coordinate value. The constant **iron.BoundaryConditionsTypes.FIXED** indicates that the value needs to be fixed to a certain value given in the argument following this constant. 

When solverEquations.BoundaryConditionsCreateFinish() is called OpenCMISS will construct the solver matrices and vectors.

In [21]:
# Prescribe boundary conditions (absolute nodal parameters)
boundaryConditions = iron.BoundaryConditions()
solverEquations.BoundaryConditionsCreateStart(boundaryConditions)
if model == 1:
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,1,X,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,3,X,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,5,X,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,7,X,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,2,X,iron.BoundaryConditionsTypes.FIXED,0.5)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,4,X,iron.BoundaryConditionsTypes.FIXED,0.5)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,6,X,iron.BoundaryConditionsTypes.FIXED,0.5)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,8,X,iron.BoundaryConditionsTypes.FIXED,0.5)

    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,1,Y,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,2,Y,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,5,Y,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,6,Y,iron.BoundaryConditionsTypes.FIXED,0.0)

    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,1,Z,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,2,Z,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,3,Z,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,4,Z,iron.BoundaryConditionsTypes.FIXED,0.0)

    p = -2.*-0.1056E+01

elif model == 2:
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,1,X,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,3,X,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,5,X,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,7,X,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,2,X,iron.BoundaryConditionsTypes.FIXED,0.25)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,4,X,iron.BoundaryConditionsTypes.FIXED,0.25)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,6,X,iron.BoundaryConditionsTypes.FIXED,0.25)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,8,X,iron.BoundaryConditionsTypes.FIXED,0.25)

    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,1,Y,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,2,Y,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,5,Y,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,6,Y,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,3,Y,iron.BoundaryConditionsTypes.FIXED,0.25)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,4,Y,iron.BoundaryConditionsTypes.FIXED,0.25)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,7,Y,iron.BoundaryConditionsTypes.FIXED,0.25)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,8,Y,iron.BoundaryConditionsTypes.FIXED,0.25)

    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,1,Z,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,2,Z,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,3,Z,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,4,Z,iron.BoundaryConditionsTypes.FIXED,0.0)

    p = -2.*-0.6656E+00

elif model == 3:
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,1,X,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,3,X,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,5,X,iron.BoundaryConditionsTypes.FIXED,0.5)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,7,X,iron.BoundaryConditionsTypes.FIXED,0.5)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,2,X,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,4,X,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,6,X,iron.BoundaryConditionsTypes.FIXED,0.5)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,8,X,iron.BoundaryConditionsTypes.FIXED,0.5)

    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,1,Y,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,2,Y,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,5,Y,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,6,Y,iron.BoundaryConditionsTypes.FIXED,0.0)

    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,1,Z,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,2,Z,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,3,Z,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,4,Z,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,5,Z,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,6,Z,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,7,Z,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,8,Z,iron.BoundaryConditionsTypes.FIXED,0.0)

    p = -2.*-0.1450E+01

elif model == 4:
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,1,X,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,3,X,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,6,X,iron.BoundaryConditionsTypes.FIXED,0.5)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,8,X,iron.BoundaryConditionsTypes.FIXED,0.5)

    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,1,Y,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,2,Y,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,5,Y,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,6,Y,iron.BoundaryConditionsTypes.FIXED,0.0)

    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,1,Z,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,3,Z,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,6,Z,iron.BoundaryConditionsTypes.FIXED,0.5)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,8,Z,iron.BoundaryConditionsTypes.FIXED,0.5)

    p = -2.*-0.1056E+01

elif model == 5:
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,1,X,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,3,X,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,5,X,iron.BoundaryConditionsTypes.FIXED,0.5)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,7,X,iron.BoundaryConditionsTypes.FIXED,0.5)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,2,X,iron.BoundaryConditionsTypes.FIXED,0.25)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,4,X,iron.BoundaryConditionsTypes.FIXED,0.25)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,6,X,iron.BoundaryConditionsTypes.FIXED,0.75)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,8,X,iron.BoundaryConditionsTypes.FIXED,0.75)

    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,1,Y,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,2,Y,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,5,Y,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,6,Y,iron.BoundaryConditionsTypes.FIXED,0.0)

    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,1,Z,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,2,Z,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,3,Z,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,4,Z,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,5,Z,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,6,Z,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,7,Z,iron.BoundaryConditionsTypes.FIXED,0.0)
    boundaryConditions.AddNode(dependentField,iron.FieldVariableTypes.U,1,1,8,Z,iron.BoundaryConditionsTypes.FIXED,0.0)

    p = -2.*-0.1000E+01

solverEquations.BoundaryConditionsCreateFinish()


### 19. Solving the problem
After our problem solver equations have been fully defined we are now ready to solve our problem. When we call the Solve method of the problem it will loop over the control loops and control loop solvers to solve our problem:

In [22]:

# Solve the problem
problem.Solve()


### 20. Post solution data extraction and analysis.
Now that the deformation problem has been solved we can extract information for analysis purposes. First, the code snippet below copies the values stored in the dependent field into the deformedField and pressureField that were set up in steps 7 and 8 above.

In [23]:
# Copy deformed geometry into deformed field
for component in [1, 2, 3]:
    dependentField.ParametersToFieldParametersComponentCopy(
        iron.FieldVariableTypes.U,
        iron.FieldParameterSetTypes.VALUES, component,
        deformedField, iron.FieldVariableTypes.U,
        iron.FieldParameterSetTypes.VALUES, component)

# Copy pressure into pressure field
dependentField.ParametersToFieldParametersComponentCopy(
        iron.FieldVariableTypes.U,
        iron.FieldParameterSetTypes.VALUES, 4,
        pressureField, iron.FieldVariableTypes.U,
        iron.FieldParameterSetTypes.VALUES, 1)

### Examples of extracting mechanics tensor fields like F, C and E. 

In [30]:
results = {}
elementNumber = 1
xiPosition = [0.5, 0.5, 0.5]
F = equationsSet.TensorInterpolateXi(
    iron.EquationsSetDerivedTensorTypes.DEFORMATION_GRADIENT,
    elementNumber, xiPosition,(3,3))
results['Deformation Gradient Tensor'] = F

C = equationsSet.TensorInterpolateXi(
    iron.EquationsSetDerivedTensorTypes.R_CAUCHY_GREEN_DEFORMATION,
    elementNumber, xiPosition,(3,3))
results['Right Cauchy-Green Deformation Tensor'] = C

E = equationsSet.TensorInterpolateXi(
    iron.EquationsSetDerivedTensorTypes.GREEN_LAGRANGE_STRAIN,
    elementNumber, xiPosition,(3,3))
results['Green-Lagrange Strain Tensor'] = E

### $I_1$, $I_2$, $I_3$

In [31]:
I1=numpy.trace(C)
I2=0.5*(numpy.trace(C)**2.-numpy.tensordot(C,C))
I3=numpy.linalg.det(C)
results['Invariants'] = [I1, I2, I3]

TC = equationsSet.TensorInterpolateXi(
    iron.EquationsSetDerivedTensorTypes.CAUCHY_STRESS,
    elementNumber, xiPosition,(3,3))
results['Cauchy Stress Tensor'] = TC

# Output of Second Piola-Kirchhoff Stress Tensor not implemented. It is
# instead, calculated from TG=J*F^(-1)*TC*F^(-T), where T indicates the
# transpose fo the matrix.
#TG = equationsSet.TensorInterpolateXi(
#    iron.EquationsSetTensorEvaluateTypes.SECOND_PK_STRESS,
#    elementNumber, xiPosition,(3,3))
#J=1. #Assumes J=1
TG = numpy.dot(numpy.linalg.inv(F),numpy.dot(
        TC,numpy.linalg.inv(numpy.matrix.transpose(F))))
results['Second Piola-Kirchhoff Stress Tensor'] = TG

# Note that the hydrostatic pressure value is different from the value quoted
# in the original lab instructions. This is because the stress has been
# evaluated using modified invariants (isochoric invariants)
#p = dependentField.ParameterSetGetElement(
#    iron.FieldVariableTypes.U,
#    iron.FieldParameterSetTypes.VALUES,elementNumber,4)
results['Hydrostatic pressure'] = p


### Clean up

In [32]:

problem.Destroy()
coordinateSystem.Destroy()
region.Destroy()
basis.Destroy()


CMFEError: 

## Additional exercises

### Changing boundary conditions

Change the value of **model** and re-run the notebook to see the effect of altered boundary conditions. 

### Modifying the code to model anisotropic materials. 
To capture the effect of anisotropic microstructural organisation two things need to be changed: (i) you need to mathematically describe the microstructure organisation within the fibre field; (ii) you need to choose an appropraite constitutive equation that captures the stress-strain relationship in the different independent microstructural directions. 

### Modifying the code to model heterogeneity in tissue properties
Heterogeneity of a tissue can be represented by assigning different values of the constitutive equation parameters in different elements within the OpenCMISS-Iron field dedicated to defining the constitutive equations parameters. In this example feel free to test this capability by assigning a different set of mooney rivlin values in a subset of the elements. You can use this code snippet below (remove the hash) to change the value at a particular node. Note, to really make a difference you should set a different value at all the nodes of the set of elements which represent a particular region of the tissue.

In the code snippet
* **value** defines the value to be set for a chosen material field parameter
* **componentNumber** is the component of the material field to be set at the node
* **userNodeNumber** defines the node number at which the material field parameter is to be set to **value**
* **versionNumber** is a reference to whether you have duplicate nodes that are sometimes useful for meshes of complex geometries. For now just now that the default value should be 1. 

In [ ]:
#materialField.ParameterSetUpdateNodeDP(iron.FieldVariableTypes.U, iron.FieldParameterSetTypes.VALUES, versionNumber, derivativeNumber, userNodeNumber, componentNumber, value)